In this notebook, different statistics of the detected ramp planes will be analyzed and visualized.<br>
Also a score will be calculated to determine how well the algorithm works.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
# Load offline ramp detection algorithm
from lidar_ramp_offline import VisualDetection
# Load methods to extract data from rosbags
from getData import unpack_bag, synchronize_topics
# Load custom plots
from plot_library_lidar import PlotLib

pio.templates.default = "plotly_dark"
# %load_ext blackcellmagic


#### **The following values need to be adjusted for each rosbag**


In [ ]:
# Rosbag path
bag_path = "/home/user/rosbags/final/slam/u_s2c_half_odom_hdl.bag"

# ROS topics
odom_topic = "/odom"
lidar_topic = "/velodyne_points"

# Ground truth coordinates of ramp (measured by using globalmap points)
x_range, y_range = [[42, 56], [-2.2, 2]]

Run algorithm with rosbag data.<br>
Store many different calculated metrics generated by the algorithm in a pandas data frame (to allow a faster plotting).


In [ ]:
def extract_data(bag_path):
    # Odometer from hdl_slam
    pose_async, t_pose = unpack_bag(bag_path, odom_topic, 'hdl_odom')
    # Velodyne point cloud
    lidar_async, t_lidar = unpack_bag(bag_path, lidar_topic)

    # Synchronize both topics (odom usually is behind lidar)
    pose, lidar = synchronize_topics(pose_async, t_pose, lidar_async, t_lidar)
    return lidar, pose

def run_the_algorithm(lidar, pose, ramp_start_x):
    """Perform ramp detection on data"""
    # Create instance of class (using standard parameters):
    vd = VisualDetection(ramp_start_x)
    # Lists to fill, will contain entry for each plane
    planes = []
    ramp_stats = []
    true_dist = []
    all_points = []
    for i in range(len(lidar)):
        plane_points, data, pc_whole = vd.spin(lidar[i], pose[i])
        planes.append(plane_points)
        ramp_stats.append(data)
        all_points.append(pc_whole)
        # True distance to start of ramp
        true_dist.append(ramp_start_x - pose[i].position.x)
    true_dist = np.array(true_dist)
    return planes, ramp_stats, true_dist, all_points

def convert_to_df(planes, ramp_stats):
    """Add pointcloud to df each time a ramp has been detected"""
    # Remove empty lists (when no ramp has been detected)
    # because information such as angle, width of ramp etc
    # are only stored when a ramp is detected, 0 otherwise
    ramp_arrays = [x for x in planes if not isinstance(x, list)]
    ramp_stats = [x for x in ramp_stats if x != []]
    # Get indices where ramp has been detected
    ramp_indices = [i for i, v in enumerate(planes) if not isinstance(v, list)]

    # Convert list to dictionary
    dic = []
    for i, arr in enumerate(ramp_arrays):
        for j, point in enumerate(arr):
            dic.append(
                {
                    "sampleIdx": i,
                    "pointIdx": j,
                    "x": point[0],
                    "y": point[1],
                    "z": point[2],
                }
            )
    # And finally to pandas data frame
    df = pd.DataFrame(dic)
    # Reorder columns
    df = df[["sampleIdx", "pointIdx", "x", "y", "z"]]
    return df, ramp_stats, ramp_indices

def ground_truth_check(df, ramp_stats, x_range, y_range):
    """Check if ?"""
    # Check if a point lies within ramp region
    lies_inside = []
    for i, x in enumerate(df["x"]):
        if x_range[0] < x < x_range[1]:
            if y_range[0] < df["y"][i] < y_range[1]:
                # True if x and y coordinate inside region
                lies_inside.append(True)
            else:
                lies_inside.append(False)
        else:
            lies_inside.append(False)
    # Add column (bool: if point lies in region) to data frame
    df["inlier"] = lies_inside

    # Calculate how many points of each sample lie in ramp region
    true_inliers = []
    samples_num = df.sampleIdx.max() + 1
    for i in range(samples_num):
        # Bool list of inliers and outliers of sample
        bool_lst = df[df["sampleIdx"] == i]["inlier"]
        # Percentage of inliers of sample
        true_inliers.append(sum(bool_lst) / float(len(bool_lst)))

    # New dataframe with stats for each frame
    # Structure reminder of ramp_stats: [angle, width, dist, true_dist]
    dic = []
    for i in range(samples_num):
        dic.append(
            {
                "sampleIdx": i,
                "TrueInliers": true_inliers[i],
                "Angle": ramp_stats[i][0],
                "Width": ramp_stats[i][1],
                "Dist": ramp_stats[i][2],
                "TrueDist": ramp_stats[i][3],
            }
        )
    # Convert dictionary to dataframe
    df_stats = pd.DataFrame(dic)
    # Reorder columns
    df_stats = df_stats[
        ["sampleIdx", "TrueInliers", "Angle", "Width", "Dist", "TrueDist"]
    ]
    return df_stats

lidar, pose = extract_data(bag_path)
planes, ramp_stats, true_dist, all_points = run_the_algorithm(lidar, pose, x_range[0])
df, ramp_stats, ramp_indices = convert_to_df(planes, ramp_stats)
df_stats = ground_truth_check(df, ramp_stats, x_range, y_range)

## Score calculation

To calculate the score, a range has to be specified in which the lidar should be able to detect the ramp. Looking at the histogram, the range between 4 and 10 m before the ramp seems reasonable.

Now filter all the data, such that only samples in the range of 4 to 10 m before the ramp are used.


In [ ]:
def calc_score(min_dist, max_dist):
    # Sensitivity
    # How many sample were recorded in given range
    expected_detections = len(
        true_dist[(min_dist < true_dist) & (true_dist < max_dist)]
    )
    # How many samples were identified as ramp in given range
    actual_detections = len(
        df_stats["TrueDist"][
            (min_dist < df_stats["TrueDist"]) & (
                df_stats["TrueDist"] < max_dist)
        ]
    )
    # Calculate ratio
    sensitivity = float(actual_detections) / expected_detections
    print("Out of {} recorded samples {} were detected in the range of {} to {}m before the ramp.".format(
        expected_detections, actual_detections, min_dist, max_dist))
    print("Resulting in a score of {:.2f}%\n".format(sensitivity * 100))

    # Inlier score
    f = df_stats["TrueInliers"][
        (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
    ]
    try:
        score2 = sum(f) / len(f)
    except ZeroDivisionError:
        score2 = 0
    print("Of the {} detected planes {:.2f}% inlier points were actually inside the ramp region.\n".format(
        actual_detections, score2 * 100))

    # False positives?
    f = df_stats["TrueInliers"][
        (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
    ]
    thresh = 0.5
    true_detections = sum(f > thresh)
    try:
        score3 = float(true_detections) / len(f)
    except ZeroDivisionError:
        score3 = 0
    print("Of the {} detected planes {} had at least {}% of points inside the ramp region.".format(
        actual_detections, true_detections, int(thresh * 100)))
    print("Resulting in a score of {:.2f}%\n\n".format(score3 * 100))


calc_score(4, 10)
calc_score(0, 100)

expected_detections = []
actual_detections = []
# Split dataset in 1m intervals
for i in range(15, -5, -1):
    # Filter distance to ramp (from odom data) in 1m intervals
    expected_detections.append(
        len(true_dist[(i < true_dist) & (true_dist < i + 1)]))
    # Get corresponding distance estimations
    actual_detections.append(
        len(
            df_stats["TrueDist"][
                (i < df_stats["TrueDist"]) & (df_stats["TrueDist"] < i + 1)
            ]
        )
    )
# Create pandas data frame
df_eval = pd.DataFrame()
# Distance to ramp e.g. a value of 15 means interval from 16m to 15m
df_eval["distToRamp"] = range(15, -5, -1)
df_eval["expectedDetections"] = expected_detections
df_eval["actualDetections"] = actual_detections

# Plots


In [ ]:
# Load my plot library
plt = PlotLib(df, df_stats, df_eval, x_range, y_range, all_points, ramp_indices)

**How was this plot created:**

- Calculate distance to start of ramp using odometry data from hdl_slam
- Calculate how many lidar samples were recorded during a 1m drive (e.g. car took 0.5s for 1m $\rightarrow$ lidar took 5 samples (because $f_\text{Lidar}=$ 10Hz))
- Calculate how many samples of them were actually identified as a ramp by the algorithm
- Represent both values in a bar plot
- Value at bar represents the lower range (e.g. bar at 15 means 16m to 15m)

**What to take from this plot:**<br>

- At which distance to the ramp is the lidar most reliable?<br>
- $\rightarrow$ Range between 10m to 4m seems to be the most reliable


In [ ]:
plt.bar_plot()

**How was this plot created:**

- Every time a ramp is detected certain metrics such as angle, width, distance are estimated by the algorithm
- Compare predictions to the true values, true values were measured by hand
- Calculate some stats like standard deviation or average error and add them
- Represent values as a line plot

**What to take from this plot:**

- (Lars wanted it)
- Distance to ramp:
  - Distance is estimated quite well up to about 3m before the ramp
  - There does seem to be a slight offset (manual measurement might have been wrong)
- Angle:
  - Quite a bit of variance at far and close distance, but very small deviation in the range 9m to 3m
  - Because ramp does not have a constant angle, angle estimation of 4° etc. at the beginning of the ramp might be correct
- Width:
  - Mhm...


In [ ]:
plt.distance_estimation().show()
plt.angle_estimation(6).show()
plt.width_estimation(2.9).show()

**How was this plot created:**

- Every time the algorithm detects a ramp, the coordinates of the inliers are stored
- But often times not all points in ramp region are detected
- Display points detected by algorithm in one color and all the other points recorded by the lidar in another color
- Add an area of the true ramp region (measured by hand)
- Calculate percentage of inliers how lie in ramp region
- Do this for every sample which was detected as ramp
- Add a slider (just to show off)

**What to take from this plot:**

- (Plotly is cool)
- Only very few "lines" are thrown on the ground / on to the ramp
- Distances of over 2m between two lines are common (especially at higher distances)


In [ ]:
plt.animation_only_detections()

**How was this plot created:**

- Similar to previous plot
- For every recorded lidar sample all points are displayed
- Add an area of the true ramp region (measured by hand)

**What to take from this plot:**

- Just gives an idea of what the lidar sees, might be useful to optimize mount pitch angle
- Also might give information, about why some samples are not recognized as ramp, eventhough they should be


In [ ]:
fig = go.Figure()
# Add traces, one for each slider step
for step in range(len(all_points)):
    fig.add_trace(
        go.Scatter(
            visible=False,
            mode="markers",
            x=all_points[step][:, 0],
            y=all_points[step][:, 1],
            name="All lidar points",
        )
    )
fig.data[0].visible = True
fig.add_shape(
    type="rect",
    x0=x_range[0],
    x1=x_range[1],
    y0=y_range[0],
    y1=y_range[1],
    fillcolor="red",
    opacity=0.2,
)
# Set static axes limits
fig.update_xaxes(range=x_fixed)
fig.update_yaxes(range=y_fixed)

# Create and add slider
steps = []
for i in range(len(fig.data)):
    ramp_det = "A" if i in ramp_indices else "No"
    step = dict(
        method="update",
        args=[
            {"visible": [False] * len(fig.data)},
            {
                "title": "{:.2f}m infront of ramp. {} ramp has been detected".format(
                    #   true_inliers[i]*100, df_stats.iloc[i,-1])}],  # layout attribute
                    true_dist[i],
                    ramp_det,
                )
            },
        ],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(active=0, currentvalue={
                "prefix": "Deteced plane: "}, steps=steps)]
fig.update_layout(
    sliders=sliders,
    xaxis_title="Global x coor [m]",
    yaxis_title="Global y coor [x]",
    showlegend=True,
)
fig.show()


**How was this plot created:**

- Every time the algorithm detects a ramp, the coordinates of the inlier points are saved
- Put all point coordinates of every detection together
- Add an area of the true ramp region (measured by hand)
- Display as a scatter plot and also as a heatmap

**What to take from this plot:**

- Shows which points are most commonly detected
- Majority of points are inside ramp region
- ?


In [ ]:
plt.all_detected_points().show()
plt.all_detected_points(show_as_heatmap=True).show()

**How was this plot created:**

- Every time a ramp is detected certain metrics such as angle, width, distance are estimated by the algorithm
- Because the inliers of a detected ramp do not always lie in the ramp region, split the dataset in "good" and "bad" sets
- "good" sets are samples, where more than 70% of inliers lie in the ramp region, whereas less than 30% do so in a "bad" sample
- For each set display the metrics as a histogram

**What to take from this plot:**

- Helps to selected a good metric, which can be used as an constraint in the detection
- Look for metrics with a low variance in the "good" dataset $\rightarrow$ potentially good metric
- Look for metrics with a high variance in the "good" dataset $\rightarrow$ potentially bad metric
- Look for metrics with a lwo variance in the "bad" dataset $\rightarrow$ could be used as an inverted condition
- Angle and width seem to be fairly good metrics (condition ranges of both metrics were already reduced, was more obvious before)


In [ ]:
# Stats for good detection
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Histogram(x=df_stats_good["Angle"], nbinsx=10, name="Angle"), row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_good["Dist"], nbinsx=10, name="Distance"), row=1, col=2
)
fig.add_trace(
    go.Histogram(x=df_stats_good["Width"], nbinsx=10, name="Width"), row=2, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_good["TrueInliers"],
                 nbinsx=20, name="True Inliers"),
    row=2,
    col=2,
)
fig.update_layout(
    title="Couple of stats of well detected ramp planes, which lie > 70% in the desired area"
)
fig.show()

# Stats for bad detection
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Histogram(x=df_stats_bad["Angle"], nbinsx=10, name="Angle"), row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_bad["Dist"], nbinsx=10, name="Distance"), row=1, col=2
)
fig.add_trace(
    go.Histogram(x=df_stats_bad["Width"], nbinsx=10, name="Width"), row=2, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_bad["TrueInliers"],
                 nbinsx=20, name="True Inliers"),
    row=2,
    col=2,
)
fig.update_layout(
    title="Couple of stats of badly detected ramp planes, which lie < 30% in the desired area"
)
fig.show()
